In [4]:
#default_exp hydranet

# HydraNet
> Inspired from Tesla but mini

In [11]:
#export
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from hydranet.layers import *

## Model

In [12]:
#export
class DynamicUnet(nn.Module):
    """A Unet that take almost any backbone from timm"""
    def __init__(self, backbone="mobilenetv2_100", dim=256):
        super().__init__()
        self.encoder = timm.create_model(backbone, pretrained=True, features_only=True)

        # passing dummy tensor to get sizes
        dummy_tensor = torch.rand([1, 3, 64, 64])
        features = self.encoder(dummy_tensor)
        ch_sizes = [list(f.shape)[1] for f in features][::-1]
        self.upsample_blocks = nn.ModuleList()

        self.mid_conv = ConvLayer(ch_sizes[0], dim, 3)

        for i, ch_size in enumerate(ch_sizes[1:]):
            self.upsample_blocks.append(UnetBlock(dim, ch_size, out_channels=dim, use_groups=(i==(len(features)-2))))
        

    def forward(self, x):
        input_shape = x.shape

        # features reversed in order
        features = self.encoder(x)[::-1]

        # put last feature on dim of the model
        x = self.mid_conv(features[0])

        # upsample blocks with shortcurts from the sides
        for f, ublock in zip(features[1:], self.upsample_blocks):
            x = ublock(f, x)
        x = F.interpolate(x, size=input_shape[-2:], mode="bilinear", align_corners=False)
        return x

In [13]:
#export
class HydraNet(nn.Module):
    def __init__(self, backbone="mobilenetv2_100", hidden_dim=256, num_classes=21):
        super().__init__()
        self.backbone = DynamicUnet(backbone, dim=hidden_dim)
        
        # A simple segmentation head
        self.segmentation_head = nn.Sequential(
            ConvLayer(hidden_dim, hidden_dim),
            nn.Conv2d(hidden_dim, num_classes, kernel_size=1, bias=False))
        
    def forward(self, x):
        
        backbone_out = self.backbone(x)

        return self.segmentation_head(backbone_out)

In [14]:
model = HydraNet()

In [15]:
model(torch.rand(1,3,64,64)).shape

torch.Size([1, 21, 64, 64])

## Export -

In [16]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_data.ipynb.
Converted 02_layers.ipynb.
Converted 03_hydranet.ipynb.
Converted 04_trainer.ipynb.
